In [141]:
#RECOMMENDATION ENGINE

Note: you may need to restart the kernel to use updated packages.


In [238]:
#IMPORT LIBRARIES
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
 #categorical data into vectors
from sklearn.metrics.pairwise import cosine_similarity     #compare the genres with one another
from ipywidgets import *  #interactive widgetss


In [219]:
#import the csv files
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')

In [220]:
#explore the data files
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [221]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [222]:
#null values
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10329 non-null  int64 
 1   title    10329 non-null  object
 2   genres   10329 non-null  object
dtypes: int64(1), object(2)
memory usage: 242.2+ KB


In [223]:
movies.shape

(10329, 3)

In [224]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105339 entries, 0 to 105338
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105339 non-null  int64  
 1   movieId    105339 non-null  int64  
 2   rating     105339 non-null  float64
 3   timestamp  105339 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.2 MB


In [225]:
#null values are not present

In [226]:
ratings.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [227]:
#different genres available
movies['genres']=movies['genres'].str.split("|")

In [228]:
movies2=movies.explode('genres')
movies2.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy


In [229]:
#unique genres available
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

In [230]:
movies2['genres'].nunique()

20

In [231]:
movies2 = movies2[movies2['genres'] != "(no genres listed)"]


In [232]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir'], dtype=object)

In [233]:
movies2['genres'].nunique()

19

In [159]:
#group the df based on genres and get the average rationgs for ecah genre
#merge movies2 and ratings
merged_data=pd.merge(ratings,movies2,on=['movieId'],how='inner')
merged_data

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime
1,1,16,4.0,1217897793,Casino (1995),Drama
2,9,16,4.0,842686699,Casino (1995),Crime
3,9,16,4.0,842686699,Casino (1995),Drama
4,12,16,1.5,1144396284,Casino (1995),Crime
...,...,...,...,...,...,...
281892,668,140098,2.5,1450415424,Runoff (2015),Drama
281893,668,140816,2.5,1443288791,Tangerine (2015),Comedy
281894,668,140816,2.5,1443288791,Tangerine (2015),Drama
281895,668,142488,4.0,1451535844,Spotlight (2015),Thriller


In [160]:
popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
popularity.columns=["Genres","Title","Average Ratings","Number of Ratings"]
popularity

,Genres,Title,Average Ratings,Number of Ratings
0,Action,'71 (2014),3.500000,1
1,Action,'Hellboy': The Seeds of Creation (2004),3.000000,1
2,Action,10 to Midnight (1983),2.500000,1
3,Action,12 Rounds (2009),2.875000,4
4,Action,13 Assassins (Jûsan-nin no shikaku) (2010),3.500000,5
...,...,...,...,...
23093,Western,Wyatt Earp (1994),3.200000,30
23094,Western,Young Guns (1988),3.375000,36
23095,Western,Young Guns II (1990),3.083333,12
23096,Western,Young Ones (2014),2.000000,1


In [161]:
#genre=action
#threshold=50
#top=7
popularity[(popularity['Genres']=='Action')&(popularity['Number of Ratings']>=50)].sort_values(by='Average Ratings',ascending=False).head(7)

,Genres,Title,Average Ratings,Number of Ratings
1179,Action,Princess Mononoke (Mononoke-hime) (1997),4.384615,52
1076,Action,North by Northwest (1959),4.273973,73
975,Action,"Matrix, The (1999)",4.264368,261
1433,Action,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
1331,Action,Seven Samurai (Shichinin no samurai) (1954),4.217742,62
1199,Action,Raiders of the Lost Ark (Indiana Jones and the...,4.212054,224
747,Action,Inception (2010),4.189320,103


In [162]:
#popularity recommender system
def TopNPopularMovies(genre,threshold,topN):
    popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
    popularity.columns=["Genres","Title","Average_Ratings","Number_of_Ratings"]
    
    #filter the data
    topNrecommendations=popularity[(popularity['Genres']==genre)&(popularity['Number_of_Ratings']>=threshold)].sort_values(by='Average_Ratings',ascending=False).head(7)

    #output
    topNrecommendations['Sno.']=range(1,len(topNrecommendations)+1)
    topNrecommendations.index=range(0,len(topNrecommendations))
    topNrecommendations.columns=['Genres','Movie Title','Average Movie Rating','Number of Reviews','Sno.']
    return topNrecommendations[['Sno.','Movie Title','Average Movie Rating','Number of Reviews']]

In [163]:
#test case
genre='Adventure'
threshold=50
topN=8
TopNPopularMovies(genre=genre,threshold=threshold,topN=topN)

,Sno.,Movie Title,Average Movie Rating,Number of Reviews
0,1,Princess Mononoke (Mononoke-hime) (1997),4.384615,52
1,2,Monty Python and the Holy Grail (1975),4.301948,154
2,3,North by Northwest (1959),4.273973,73
3,4,Spirited Away (Sen to Chihiro no kamikakushi) ...,4.236111,72
4,5,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
5,6,Seven Samurai (Shichinin no samurai) (1954),4.217742,62
6,7,Raiders of the Lost Ark (Indiana Jones and the...,4.212054,224


In [164]:
#content Based Recommender System

In [165]:
#tfidf vectorizer>>>genres

In [166]:
movies2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy


In [167]:
movies3=movies2.groupby('title').agg({'genres':lambda x:" ".join(list(x))}).reset_index()

In [168]:
movies3.head()

,title,genres
0,'71 (2014),Action Drama Thriller War
1,'Hellboy': The Seeds of Creation (2004),Action Adventure Comedy Documentary Fantasy
2,'Round Midnight (1986),Drama Musical
3,'Til There Was You (1997),Drama Romance
4,"'burbs, The (1989)",Comedy


In [169]:
#instance of tfidf vectorizer>>>>>used to apply the function upon genres and generate vectors
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,3),stop_words='english')
tf

TfidfVectorizer(ngram_range=(1, 3), stop_words='english')

In [170]:
tf_matrix=tf.fit_transform(movies3['genres'])

In [171]:
cosine_sim=cosine_similarity(tf_matrix,tf_matrix)

In [172]:
cosine_sim

array([[1.        , 0.02677945, 0.02931913, ..., 0.10229517, 0.        ,
        0.        ],
       [0.02677945, 1.        , 0.        , ..., 0.03626651, 0.02411583,
        0.02863994],
       [0.02931913, 0.        , 1.        , ..., 0.        , 0.        ,
        0.35526663],
       ...,
       [0.10229517, 0.03626651, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.02411583, 0.        , ..., 0.        , 1.        ,
        0.07090711],
       [0.        , 0.02863994, 0.35526663, ..., 0.        , 0.07090711,
        1.        ]])

In [173]:
indices=pd.Series(movies3.index,index=movies3['title'])
indices

title
'71 (2014)                                           0
'Hellboy': The Seeds of Creation (2004)              1
'Round Midnight (1986)                               2
'Til There Was You (1997)                            3
'burbs, The (1989)                                   4
                                                 ...  
loudQUIETloud: A Film About the Pixies (2006)    10315
xXx (2002)                                       10316
xXx: State of the Union (2005)                   10317
¡Three Amigos! (1986)                            10318
À nous la liberté (Freedom for Us) (1931)        10319
Length: 10320, dtype: int64

In [174]:
index=indices["'71 (2014)"]

In [175]:
scores=list(enumerate(cosine_sim[index]))

In [176]:
matched=sorted(scores,key=lambda x:x[1],reverse=True)[:5]

In [177]:
matched=[i[0] for i in matched]
matched

[0, 596, 858, 3839, 4407]

In [178]:
movies3.iloc[matched]

,title,genres
0,'71 (2014),Action Drama Thriller War
596,Army of Shadows (L'armée des ombres) (1969),Action Drama Thriller War
858,Battle Royale 2: Requiem (Batoru rowaiaru II: ...,Action Drama Thriller War
3839,Green Zone (2010),Action Drama Thriller War
4407,"Hurt Locker, The (2008)",Action Drama Thriller War


In [189]:
#final function
def recommendation_genre(movie_df,similarity_matrix,movie_title,topN):
    #indices for all movies
    indices=pd.Series(movies3.index,index=movies3['title'])
    #index of target movie
    index=indices[movie_title]
    #cosine similarity scores
    cosine_scores=list(enumerate(similarity_matrix[index]))
    cosine_scores=sorted(cosine_scores,key=lambda x:x[1],reverse=True)[1:topN+2]
    #extract the matching movies
    matched=[i[0] for i in cosine_scores]
    matching_df=movie_df.iloc[matched]

    #filter out the target movie
    matching_df=matching_df[matching_df['title']!=movie_title]

    #output
    matching_df.rename(columns={'title':'Movie Title'},inplace=True)
    matching_df['Sno.']=range(1,len(matching_df)+1)
    matching_df.index=range(0,len(matching_df))
    return matching_df[['Sno.','Movie Title']].head(topN)
    

In [180]:
#test case
recommendation_genre(movie_df=movies3,similarity_matrix=cosine_sim,movie_title='Shrek the Third (2007)',topN=10)

,Sno.,Movie Title
0,1,Antz (1998)
1,2,Asterix and the Vikings (Astérix et les Viking...
2,3,"Boxtrolls, The (2014)"
3,4,DuckTales: The Movie - Treasure of the Lost La...
4,5,"Emperor's New Groove, The (2000)"
5,6,"Monsters, Inc. (2001)"
6,7,"Tale of Despereaux, The (2008)"
7,8,Toy Story (1995)
8,9,Toy Story 2 (1999)
9,10,Turbo (2013)


In [181]:
#interactie widgets

In [190]:
#popularity
#inputs
genres=Dropdown(options=list(set(movies2['genres'])),description="Genres",style={"description_width":'initial'})
num_reviews=IntText(description="Minimum Reviews",style={"description_width":'initial'})
num_recommendations_1=IntText(description="Number of Recommendations",style={"description_width":'initial'})

#tabs
b1=Button(description="RECOMMEND ME",style={"description_width":'initial'})
h1=HBox([num_reviews,num_recommendations_1])
popularity_tab=VBox([genres,h1,b1])

#content based system
title=Textarea(description="Movie Title",style={"description_width":'initial'})
num_recommendations_2=IntText(description="Number of Recommendations",style={"description_width":'initial'})

#tabs
b2=Button(description="RECOMMEND ME",style={"description_width":'initial'})
h2=HBox([title,num_recommendations_2])
content_tab=VBox([h2,b2])


#creating final tabs
tabs=[popularity_tab,content_tab]
wid=widgets.Tab(tabs)

#set title to the tabs
names=['Popularity Based Recommendations','Content Based Recommendations']
[wid.set_title(i,title) for i,title in enumerate(names)]

display(wid)

In [212]:
#setting up the events t respond when clicked upon
#popularity
def b1_clicked(b):
    global output
    output=TopNPopularMovies(genre=genre.value,threshold=num_reviews.value,topN=num_recommendations_1.values)
b1.on_click(b1_clicked)

#content
def b2_clicked(b):
    global output
    result=recommendation_genre(movie_df=movies3,similarity_matrix=cosine_sim,movie_title=title.value,topN=num_recommendations_2.value)
    output=result
b2.on_click(b2_clicked)

In [207]:
display(wid)

In [214]:
output

,Sno.,Movie Title
0,1,Antz (1998)
1,2,Asterix and the Vikings (Astérix et les Viking...
2,3,"Boxtrolls, The (2014)"
3,4,DuckTales: The Movie - Treasure of the Lost La...
4,5,"Monsters, Inc. (2001)"
5,6,Shrek the Third (2007)
6,7,"Tale of Despereaux, The (2008)"
7,8,Toy Story (1995)
8,9,Toy Story 2 (1999)
9,10,Turbo (2013)
